# **Congerssional Voting First Model Atempt**
## *Bartosz Sawicki, Hubert Ruczyński*

In [ ]:
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
plt.style.use('ggplot')

from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier # Inna paczka niż sklearn!

# **Wczytanie Danych**

In [ ]:
url = 'https://api.apispreadsheets.com/api/dataset/congressional-voting/'

r = requests.get(url)
data = r.json()
df = pd.DataFrame.from_dict(data['data'], orient='columns')
df.sample(5)

# **Autorski Encoding**

In [ ]:
val = {'n':-1, '?':0, 'y':1}
party = {'democrat':2, 'republican':-2}

df_num=df.copy()
df_party = df_num['political_party'].copy()
df_num.drop('political_party', axis=1, inplace=True)

for column in df_num.columns:
  df_num[column] = df_num[column].map(val)

df_party = df_party.map(party)

df_num = pd.concat([df_num,df_party], axis=1)
df_num.head()

# **Informacje o Danych**

In [ ]:
x = df_num.drop('political_party', axis=1)
y = df_num.iloc[:,16]
x.info()
y

# **Perwszy model: SVM**


Model: https://scikit-learn.org/stable/modules/svm.html#classification



Za pierwszym razem nie dokonaliśmy selekcji zmiennych i z tego powodu otrzymaliśmy dość nieskuteczny model.

Acccuracy powyżej 0,9 może być mylące gdyż na pierwszy rzut wydaje się, że jest to wynik bardzo dobry, 
jednak po analizie której dokonywaliśmy ostatnio, wiemy, że pojedyncze głosowanie (physicians_fee)
jest w stanie przewidywać z dokładnością prawie 94%.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(x_train, y_train)
y_train_pred=clf.predict(x_train)

In [ ]:
y_pred=clf.predict(x_test)

In [ ]:
print('\ny:     ' + str(y_test[0:10]) + '\ny_pred: ' + str(y_pred[0:10]))
sum=0
y_tezt=y_test.to_numpy()
for i in range(len(y_pred)):
    if y_pred[i]==y_tezt[i]:
        sum+=1
accuracy=sum/len(y_pred)
print("validation : " + str(accuracy))
sum=0
y_trained=clf.predict(x_train)
y_trains=y_train.to_numpy()
for i in range(len(y_trained)):
    if y_trained[i]==y_trains[i]:
        sum+=1
accuracy_train=sum/len(y_trained)
print("train : " + str(accuracy_train))

Aby sprawdzić faktyczne accuracy, postanowiliśmy zapętlić wybór zbioru testowego, tworzenia i trenowania 
modelu 1000 razy, aby otrzymać średnie accuracy dla tych prób.

Ponadto wykonaliśmy selekcji zmiennych przy tym testowaniu w dwóch wariantach i okazało się, że 
lepiej wypadł ten przy mniejszej liczbie wyrzucanych kolumn.

## Uwaga!

Ilość testowanych modeli i zbiory wyrzucanych kolumn były znacznie większe w fazie pisania raportu, jednak postanowiliśmy zostawić tylko te które coś wnoszą.

In [ ]:
accumulated_accuracy=0
accumulated_accuracy_train=0
for i in range(1000):
    x = df_num.drop(['political_party', 'water_project_cost_sharing','immigration','export_administration_act_south_africa'], axis=1)
    y = df_num.iloc[:,16]
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    y_train_pred=clf.predict(x_train)
    y_pred=clf.predict(x_test)
    sum=0

    y_tezt=y_test.to_numpy()
    for i in range(len(y_pred)):
        if y_pred[i]==y_tezt[i]:
            sum+=1
    accuracy=sum/len(y_pred)
    accumulated_accuracy=accumulated_accuracy+accuracy
    sum=0
    y_trained=clf.predict(x_train)
    y_trains=y_train.to_numpy()
    for i in range(len(y_trained)):
        if y_trained[i]==y_trains[i]:
            sum+=1
    accuracy_train=sum/len(y_trained)
    accumulated_accuracy_train=accumulated_accuracy_train+accuracy_train

print("accumulated accuracy validation : " + str(accumulated_accuracy/1000))
print("accumulated accuracy train : " + str(accumulated_accuracy_train/1000))

In [ ]:
accumulated_accuracy=0
accumulated_accuracy_train=0
for i in range(1000):
    x = df_num.drop(['political_party', 'water_project_cost_sharing','immigration','export_administration_act_south_africa', 
                   'education_spending', 'synfuels_corporation_cutback','religious_groups_in_schools'], axis=1)
    y = df_num.iloc[:,16]
    x_train, x_test, y_train, y_test = train_test_split(x, y)
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    y_train_pred=clf.predict(x_train)
    y_pred=clf.predict(x_test)
    sum=0

    y_tezt=y_test.to_numpy()
    for i in range(len(y_pred)):
        if y_pred[i]==y_tezt[i]:
            sum+=1
    accuracy=sum/len(y_pred)
    accumulated_accuracy=accumulated_accuracy+accuracy
    sum=0
    y_trained=clf.predict(x_train)
    y_trains=y_train.to_numpy()
    for i in range(len(y_trained)):
        if y_trained[i]==y_trains[i]:
            sum+=1
    accuracy_train=sum/len(y_trained)
    accumulated_accuracy_train=accumulated_accuracy_train+accuracy_train
    
print("accumulated accuracy validation : " + str(accumulated_accuracy/1000))
print("accumulated accuracy train : " + str(accumulated_accuracy_train/1000))


Na autorskim kodowaniu postanowiliśmy też wykorzystać bardziej złożony model w postaci XGBClassifier'a, który
zapewnił nam o wiele wyższe accuracy od poprzedników.

In [ ]:
x = df_num.drop(['political_party', 'water_project_cost_sharing','immigration','export_administration_act_south_africa'], axis=1)
y = df_num.iloc[:,16]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=14)

In [ ]:
model=XGBClassifier(random_state=2,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    nround = 1000, # Ilość itereacji boosingowych
                    max_depth=3, # Maksymalna głębokość drzewa 
                    verbosity = 0 # Nie chcemy warningów
                    )
model.fit(x_train, y_train)
prediction_test=model.predict(x_test)
print(model.score(x_test,y_test))
print(metrics.classification_report(y_test, prediction_test))

# **Kodowanie one hot**

Aby uzyskać jak najlepszy model użyliśmy jednak one-hot encodingu, ze względu na to, że jest to lepsza wersja 
tego co sami robimy w naszym kodowaniu.


In [ ]:
df.info()


Modele działać, będą na dwóch ramkach, jedna cała, bez selekcji zmiennych i druga z odrzuceniem tych które
są w bardzo niskim stopniu skorelowane z targetem.

In [ ]:
df_dropped = df.drop(['water_project_cost_sharing','immigration','export_administration_act_south_africa'], axis=1)
X_dropped = df_dropped.iloc[:,:12]
y_dropped = df_dropped.iloc[:,13]

X = df.iloc[:,:15]
y = df.iloc[:,16]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_dropped.iloc[:,:12], df_dropped.iloc[:,13], random_state=43)

In [ ]:
import category_encoders as ce
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline

one_hot_encoder = ce.OneHotEncoder()

one_hot = one_hot_encoder.fit_transform(X,y)

one_hot_encoder_dropped = ce.OneHotEncoder()
one_hot_dropped = one_hot_encoder_dropped.fit_transform(X_dropped,y_dropped)

# **GradientBoostingClassifier**

In [ ]:
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=.05,
    max_depth=3, random_state=997)

pipe_one_hot = Pipeline(
[
    ('transformer_one_hot', one_hot_encoder),
    ('classifier', clf)
])

pipe_one_hot_dropped = Pipeline(
[
    ('transformer_one_hot', one_hot_encoder_dropped),
    ('classifier', clf)
])

In [ ]:
print(np.mean(cross_validate(pipe_one_hot, X, y, cv=11, scoring='accuracy').get('test_score')))
print(np.mean(cross_validate(pipe_one_hot_dropped, X_dropped, y_dropped, cv=11, scoring='accuracy').get('test_score')))

# **XGBClassifier**

In [ ]:
model=XGBClassifier(random_state=1,
                    learning_rate=0.01, # Szybkość "uczenia" się
                    booster='gbtree', # Jaki model wykorzystujemy (drzewo - gbtree, liniowe - gblinear)
                    nround = 1000, # Ilość itereacji boosingowych
                    max_depth=3, # Maksymalna głębokość drzewa 
                    verbosity = 0
                    )
XGB_one_hot = Pipeline(
[
    ('transformer_one_hot', one_hot_encoder),
    ('classifier', model)
])

XGB_one_hot_dropped = Pipeline(
[
    ('transformer_one_hot', one_hot_encoder_dropped),
    ('classifier', model)
])

In [ ]:
print(np.mean(cross_validate(XGB_one_hot, X, y, cv=11, scoring='accuracy').get('test_score')))
print(np.mean(cross_validate(XGB_one_hot_dropped, X_dropped, y_dropped, cv=11, scoring='accuracy').get('test_score')))

# **AdaBoostClassifier**

In [ ]:
model = AdaBoostClassifier(random_state=1,learning_rate=0.1,algorithm ='SAMME.R')
Ada_one_hot = Pipeline(
[
    ('transformer_one_hot', one_hot_encoder),
    ('classifier', model)
])

Ada_one_hot_dropped = Pipeline(
[
    ('transformer_one_hot', one_hot_encoder_dropped),
    ('classifier', model)
])

In [ ]:
print(np.mean(cross_validate(Ada_one_hot, X, y, cv=7, scoring='accuracy').get('test_score')))
print(np.mean(cross_validate(Ada_one_hot_dropped, X_dropped, y_dropped, cv=7, scoring='accuracy').get('test_score')))


## *Uwaga!*

Ze względu na istotę problemu, czyli wybór między dwoma ugrupowaniami politycznymi, jedyną interesującą nas miarą jest accuracy, gdyż dla nas jest to bez różnicy czy pomylimy się mówiąc, że republikanin jest demokratą, czy też na odwrót.

# **TO DO:**

Zamierzamy teraz skupić się na dwóch najlepiej zapowiadających się modelach, czyli AdaBoostClassifier i XGBClassifier. Chcemy poprawić ich parametry w taki sposób, aby otrzymać jak najlepsze wyniki.